# 💬 IA Generativa com RAG e Agentes

Este notebook demonstra como utilizar:
- Recuperação aumentada com geração (RAG) usando LangChain + Qdrant
- Agentes com ferramentas e memória usando LangChain

Vamos começar!

In [2]:
# ✅ Instalação de dependências
# !pip install -U langchain langchain-community openai qdrant-client duckduckgo-search tiktoken

In [4]:
from dotenv import load_dotenv

load_dotenv()

True

## 🔍 Exemplo RAG com Qdrant

In [6]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Qdrant
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

# Simulando documentos
docs = [Document(page_content="IA pode ser usada para gerar texto com base em documentos técnicos.")]

splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
chunks = splitter.split_documents(docs)

embeddings = OpenAIEmbeddings()
qdrant = Qdrant.from_documents(chunks, embeddings, location=":memory:", collection_name="meus_docs")

qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(), retriever=qdrant.as_retriever())
qa.invoke("Como a IA pode ser usada?")

{'query': 'Como a IA pode ser usada?',
 'result': 'A IA pode ser usada para uma variedade de tarefas, como reconhecimento de padrões, processamento de linguagem natural, controle de sistemas autônomos, entre outras aplicações. Ela pode ajudar a automatizar processos, otimizar sistemas e até mesmo gerar texto com base em documentos técnicos, como mencionado anteriormente.'}

## 🤖 Agente com Busca Web

In [7]:
from langchain.agents import initialize_agent, Tool
from langchain.agents.agent_types import AgentType
from langchain.memory import ConversationBufferMemory
from langchain.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()
tools = [Tool(name="Search", func=search.run, description="Busca na web")]
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
agent = initialize_agent(tools, ChatOpenAI(), agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION, memory=memory)

agent.run("Quem é o presidente do Brasil?")

/tmp/ipykernel_188480/4183391828.py:8: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
/tmp/ipykernel_188480/4183391828.py:9: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(tools, ChatOpenAI(), agent=AgentType.CONVERSA

'The current president of Brazil is Jair Bolsonaro.'